In [1]:
import pygsheets
import pprint
import json
import requests
from tqdm import tqdm
import random
import pprint
import pandas as pd
import copy
import time
import pickle
from collections import OrderedDict 
from multiprocessing import Pool


from IPython.display import display, HTML
# gc = pygsheets.authorize(client_secret='/content/drive/My Drive/11th Grade/MicroEcon/client_secret.json')
# normal='1WqHMSvF9VR6sNZsDLmOPgMColtHigLjQQBzlT_IV-6c'
# noise='1o7i3NP3DJ1p7tq8qELiB5n3qcwaGZmWr6rOQRPtzY6M'
# sheet = gc.sheet.get(normal)
#./gdrive-linux-x64 upload -r --parent 1vJtLcFMasO31eOoWOA9cDZ6KDDccJfb6 ~/pench/correlate/

In [2]:
# %gui qt
import numpy as np
import mne
import pickle
import sys
import os
import matplotlib

import matplotlib.pyplot as plt

# import vispy
# print(vispy.sys_info())
# BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(BASE_DIR)
%matplotlib inline
mne.utils.set_config('MNE_USE_CUDA', 'true')  
mne.cuda.init_cuda(verbose=True)

Now using CUDA device 0
Enabling CUDA with 23.28 GB available memory


In [3]:
bloc={
    "sync":1,
    "baseline":2,
    "stressor":3,
    "survey":4,
    "rest":5,
    "slowBreath":6,
    "paced":7
}

reverse={
    1:"sync1",
    2:"baseline1",
    3:"stressor1",
    4:"survey1",
    5:"rest1",
    6:"slowBreath1",
    7: "paced1"
}

colors={
    1:'w', #White
    2:'k',
    3:'r',
    4:'y',
    5:'m',
    6:'b',
    7:'c'
}

lines={
    'bpm':'-',
    'sdnn':'-',
    'rmssd':'-',
    'pnn50':'-'
}

#Generate blocks

blocks=[
    ("sync",6),
    ("baseline", 120),
    ("stressor", 90),
    ("survey", 10),
    ("rest", 30),
    ("slowBreath", 200),
    ("survey", 10),
    ("rest", 30),
    ("stressor", 90),
    ("survey", 10),
    ("rest", 30),
    ("paced", 200),
    ("survey", 10),
    ("rest", 30),
    ("stressor", 90),
    ("survey", 10),
    ("rest", 30),
    ("slowBreath", 200),
    ("survey", 10),
    ("rest", 30),
    ("stressor", 90),
    ("survey", 10),
    ("rest", 30),
    ("paced", 200),
    ("survey", 10),
    ("rest", 30)
]

freq_bands = dict(
    delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))

len(blocks)
timeline = [[], []]
star=0
for elem in blocks:
    timeline[0].append(star+elem[1])
    timeline[1].append(elem[0])
    star+=elem[1]
for i in range(len(timeline[0])):
    timeline[0][i]=125*timeline[0][i]
# timeline=pickle.load(open('timeline', 'rb'))

In [4]:

def createMNEObj(data, name='Empty'):
    #Create Metadata
    
    sampling_rate = 125
    channel_names = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2', 'F7', 'F8', 'F3', 'F4', 'T7', 'T8', 'P3', 'P4',
                    'time', 'bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate', 'segment_indices1', 'segment_indices2', 'block']
    channel_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
                     'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'stim']
    n_channels = len(channel_types)
    info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
    info['description'] = name
    print(info)
    transformed = []
    start=-1.0
    
    for i in range(len(data)):
        add=[]
        add=data[i][1:17]
#         print(data[i][19].keys())
        if start==-1:
            start=data[i][18].hour*3600 + data[i][18].minute*60 + data[i][18].second + data[i][18].microsecond/1000
            add.append(0.0)
        else:
            tim=data[i][18].hour*3600 + data[i][18].minute*60 + data[i][18].second + data[i][18].microsecond/1000
            add.append(tim-start)
#         add.append(str(data[i][18].hour)+':'+str(data[i][18].minute)+':'+str(data[i][18].second)+':'+str(int(data[i][18].microsecond/1000)))
#         try:
        add.append(data[i][19]['bpm'])
#         except Exception as e: 
#             print(e, i)
#             print(data[i][19])
#             print(len(data))
        
        add.append(data[i][19]['ibi'])
        add.append(data[i][19]['sdnn'])
        add.append(data[i][19]['sdsd'])
        add.append(data[i][19]['rmssd'])
        add.append(data[i][19]['pnn20'])
        add.append(data[i][19]['pnn50'])
        add.append(data[i][19]['hr_mad'])
        add.append(data[i][19]['sd1'])
        add.append(data[i][19]['sd2'])
        add.append(data[i][19]['s'])
        add.append(data[i][19]['sd1/sd2'])
        add.append(data[i][19]['breathingrate'])
        add.append(data[i][19]['segment_indices'][0])
        add.append(data[i][19]['segment_indices'][1])
        add.append(bloc[data[i][20]])
        transformed.append(np.array(add))
    
    transformed=np.array(transformed)
    print(transformed[0])
    #have to convert rows to columns to fit MNE structure
    transformed=transformed.transpose()
    print(transformed[0], transformed[1], transformed[2], transformed[3])
    print(len(transformed[0]))
    loaded=mne.io.RawArray(transformed, info)
    return loaded

In [5]:
baseFolder="/home/sean/pench/pickled-avg/"
files=[f for f in os.listdir(baseFolder) if not f.startswith('.')]

def doStuff(name):
    data=pickle.load(open('pickled-avg/'+name, 'rb'))
    raw=createMNEObj(data)
    raw.filter(0.5, 50, fir_design='firwin')
    montage = mne.channels.make_standard_montage('easycap-M1')
    raw.set_montage(montage, raise_if_subset=False)
    out = OrderedDict() 
    out['sync1']=[]
    prev='sync'
    star=0
    eegChunk=10
    def findPower(band):
        da=band[0]
        tot=0
        for channel in da:
            su=0
            for freq in channel:
                su+=freq
            avg=su/len(channel)
            tot+=su
        return tot/len(da)
    for i in range(len(data)):
        if data[i][20]!=prev:
            print(prev)
            reverse[bloc[data[i][20]]]=reverse[bloc[data[i][20]]][:-1]+str(int(reverse[bloc[data[i][20]]][-1])+1)
            out[reverse[bloc[data[i][20]]]]=[]
            theta=mne.time_frequency.psd_multitaper(raw, fmin=3, fmax=7, tmin=star/125, tmax=i/125)
            beta=mne.time_frequency.psd_multitaper(raw, fmin=15, fmax=29, tmin=star/125, tmax=i/125)
            alpha=mne.time_frequency.psd_multitaper(raw, fmin=8, fmax=14, tmin=star/125, tmax=i/125)
            thetaPower=findPower(theta)
            betaPower=findPower(beta)
            alphaPower=findPower(alpha)
            star=i
            out[reverse[bloc[data[i][20]]]].append(data[i][19]['pnn50'])
            out[reverse[bloc[data[i][20]]]].append(data[i][19]['hf'])
            out[reverse[bloc[data[i][20]]]].append(data[i][19]['bpm'])
            out[reverse[bloc[data[i][20]]]].append(data[i][19]['sdnn'])
            out[reverse[bloc[data[i][20]]]].append(data[i][19]['rmssd'])
            out[reverse[bloc[data[i][20]]]].append(thetaPower/betaPower)
            out[reverse[bloc[data[i][20]]]].append(alphaPower/thetaPower)

            prev=data[i][20]
#     f=open('correlate/'+name+'.csv', 'w')
#     for block in list(out.keys()):
#         f.write(block + ', ' + ', '.join(str(x) for x in out[block]) + '\n')
    return out
#         ', '.join(str(x) for x in out[block])

In [6]:
t=len(files)
if len(files)>32:
    t=32
p = Pool(len(files))
master=p.map(doStuff, files)

<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types

<Info | 17 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fp2, C3, C4, P7, P8, O1, O2, F7, ...
    chs : list | 33 items (EEG: 16, MISC: 16, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 5 items
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 62.5 Hz
    meas_date : NoneType | unspecified
    nchan : int | 33
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 125.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    device_info : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    helium_info : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_nam

<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)


<Info | 17 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fp2, C3, C4, P7, P8, O1, O2, F7, ...
    chs : list | 33 items (EEG: 16, MISC: 16, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 5 items
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 62.5 Hz
    meas_date : NoneType | unspecified
    nchan : int | 33
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 125.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    device_info : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    helium_info : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_nam

/home/sean/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: Warning: converting a masked element to nan.
<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: Channel names are not unique, found duplicates fo

...

<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)


<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-4-8a1b1b33d714>:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)


<Info | 17 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fp2, C3, C4, P7, P8, O1, O2, F7, ...
    chs : list | 33 items (EEG: 16, MISC: 16, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 5 items
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 62.5 Hz
    meas_date : NoneType | unspecified
    nchan : int | 33
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 125.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    device_info : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    helium_info : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_nam

/home/sean/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: Warning: converting a masked element to nan.


[-1.25328e+04 -6.62655e+03 -3.02493e+03 -7.51656e+03  8.53937e+03
 -2.94755e+03 -1.41341e+03 -5.32443e+03 -6.25165e+03 -1.43188e+03
 -1.76270e+02 -3.59141e+03  4.59985e+03 -7.25504e+03  1.94149e+03
 -3.98002e+03  0.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00
 -1.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00
 -1.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00 -1.00000e+00
 -1.00000e+00 -1.00000e+00  1.00000e+00]
[-12532.8  -12206.62 -12512.15 ... -10237.88  -9690.82 -10192.46] [-6626.55 -6254.91 -6607.89 ... -9435.52 -8809.65 -9387.58] [-3024.93 -2528.43 -2987.76 ... -3409.62 -2722.35 -3359.94] [-7516.56 -7099.61 -7482.27 ... -6282.54 -5670.7  -6242.75]
201273
Creating RawArray with float64 data, n_channels=33, n_times=201273
    Range : 0 ... 201272 =      0.000 ...  1610.176 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal b

In [7]:
master

[OrderedDict([('sync1', []),
              ('baseline2',
               [-1, -1, -1, -1, -1, 3.67860774190579, 0.47225385224849775]),
              ('stressor2',
               [-1, -1, -1, -1, -1, 1.7554100956664918, 1.99382255962371]),
              ('survey2',
               [-1, -1, -1, -1, -1, 1.9146191629933338, 2.5300875988388527]),
              ('rest2',
               [-1, -1, -1, -1, -1, 3.0824232340975795, 0.5781998689306372]),
              ('slowBreath2',
               [0.25,
                3885.38277745139,
                80.40935672514621,
                76.28394109785812,
                40.98780306383839,
                2.5574450581945953,
                0.5558319357919911]),
              ('survey3',
               [-1, -1, -1, -1, -1, 1.8492900410071424, 1.3348743857621455]),
              ('rest3',
               [-1, -1, -1, -1, -1, 1.632996271612817, 1.93042521098183]),
              ('stressor3',
               [nan,
                nan,
                na

In [8]:
f=open('correlate/master.csv', 'w')
for i in range(len(master)):
    out=master[i]
    for block in list(out.keys()):
        f.write('P' + str(i)+ ' ' + block + ', ' + ', '.join(str(x) for x in out[block]) + '\n')

In [9]:
!~/gdrive-linux-x64 upload -r --parent 1vJtLcFMasO31eOoWOA9cDZ6KDDccJfb6 ~/pench/correlate/master.csv

Uploading /home/sean/pench/correlate/master.csv
